# Load data sets

## Data set w/ technical specs

In [30]:
import pandas as pd
df1 = pd.read_csv("C:\\Users\\Jessica Tran\\Downloads\\2017_2024_GSM_scaled_final.csv")

df1 = df1[['approx_price_eur_scale', 'battery_mah_scale','ram_in_gb_scale','internal_memory_in_gb_scale','model','brand','id']]
df1.shape

(9931, 7)

### Convert to appropriate data type

In [31]:
df1.dtypes

approx_price_eur_scale         float64
battery_mah_scale              float64
ram_in_gb_scale                float64
internal_memory_in_gb_scale    float64
model                           object
brand                           object
id                              object
dtype: object

In [32]:
df1[df1.select_dtypes(include=['object']).columns] = df1.select_dtypes(include=['object']).astype('category')
#df1.head()

In [33]:
df1.dtypes

approx_price_eur_scale          float64
battery_mah_scale               float64
ram_in_gb_scale                 float64
internal_memory_in_gb_scale     float64
model                          category
brand                          category
id                             category
dtype: object

## Sales Data

In [34]:
df2 = pd.read_csv("C:\\Users\\Jessica Tran\\OneDrive\\Dokumenter\\GitHub\\phoneData\\sales\\0.Raw_data\\top 120 best-selling mobile phones.csv")
df2 = df2[['Manufacturer', 'Model','Units Sold (million )','Rank', 'Year']]
df2.shape

(120, 5)

### Convert to appropriate data type

In [35]:
df2.dtypes

Manufacturer              object
Model                     object
Units Sold (million )    float64
Rank                       int64
Year                       int64
dtype: object

In [36]:
df2[df2.select_dtypes(include=['object']).columns] = df2.select_dtypes(include=['object']).astype('category')

In [37]:
df2.dtypes

Manufacturer             category
Model                    category
Units Sold (million )     float64
Rank                        int64
Year                        int64
dtype: object

# Join datasets

Since joining the datasets are performed differently for phones before and after 2017 the process is split in two.

## Phone models before and in 2017

In [38]:
# Keep rows where 'Year' is less than or equal to 2017
df2_before = df2[df2['Year'] <= 2017].copy()

len(df2_before['Year'].unique())
(df2_before)

,Manufacturer,Model,Units Sold (million ),Rank,Year
0,Nokia,1100,250.0,1,2003
1,Nokia,1110,247.5,2,2005
2,Apple,iPhone 6 & 6 Plus,222.4,3,2014
3,Nokia,105 Series,200.0,4,2013
4,Apple,iPhone 6S & 6S Plus,174.1,5,2015
...,...,...,...,...,...
112,Nokia,N-Gage,3.0,113,2003
113,LG,G2,3.0,114,2013
114,LeTV,LeEco Le 1s,3.0,115,2016
118,Google,Pixel & Pixel XL,2.1,119,2016


### Modify model names to align with df1

In [39]:
df2_before['Model'] = df2_before['Model'].str.replace(r'^(Cookie)\((KP500)\)$', r'\2 \1', regex=True)
df2_before['Model'] = df2_before['Model'].str.replace(r'(Star/Tocco Lite) \((S5230)\)', r'\2 Star', regex=True)
df2_before['Model'] = df2_before['Model'].str.replace(r'^(Viewty)\((KU990)\)$', r'\2 \1', regex=True)

# Replace Samsung Star/Tocco Lite (S5230) with S5230 Star
df2_before['Model'] = df2_before['Model'].str.replace(r'^(.*) \((S5230)\)$', r'\2 \1', regex=True)

# Lowercase 's' for iPhone models
df2_before['Model'] = df2_before['Model'].str.replace(r'(iPhone \d+)(S)', r'\1s', regex=True)

# Add space in iPhone models and RAZR models
df2_before['Model'] = df2_before['Model'].str.replace(r'(iPhone)(\d+)', r'\1 \2', regex=True)
df2_before['Model'] = df2_before['Model'].str.replace(r'(RAZR)(\d+)', r'RAZR \2', regex=True)

# Add 'I9505' for 'Galaxy S4'
df2_before['Model'] = df2_before['Model'].str.replace(r'(^Galaxy S4$)', r'I9505 \1', regex=True)

# Add 'N7100' to 'Galaxy Note II'
df2_before['Model'] = df2_before['Model'].str.replace(r'^(Galaxy Note II)$', r'\1 N7100', regex=True)

# Add '4G' to 'ThunderBolt'
df2_before['Model'] = df2_before['Model'].str.replace(r'^(ThunderBolt)$', r'\1 4G', regex=True)

# Uppercase 'Droid Bionic' and add 'XT875'
df2_before['Model'] = df2_before['Model'].str.replace(r'^(Droid Bionic)$', lambda x: x.group(0).upper() + ' XT875', regex=True)

# Remove 'LeEco' from 'LeEco Le 1s'
df2_before['Model'] = df2_before['Model'].str.replace(r'^(LeEco) ', '', regex=True)
df2_before['Model'] = df2_before['Model'].str.replace(r'^(StarTAC)$', r'\1 75', regex=True)


# Display unique model names after all changes
(df2_before['Model'].unique())

array(['1100', '1110', 'iPhone 6 & 6 Plus', '105 Series',
       'iPhone 6s & 6S Plus', 'iPhone 5s', '3210', 'iPhone 7 & 7 Plus',
       '6600', '1200', '5230', 'E1100', 'iPhone 5',
       '2600(2610/2626/2630)', 'RAZRV3', '1600(1650/1661)', '3310(3330)',
       'iPhone 8 & 8 Plus', '1208(1209)', '1280', '6010(6020/6030)',
       'Galaxy S III & Galaxy S III Mini', 'I9505 Galaxy S4',
       '5130(5220/5310)', 'iPhone X', 'iPhone 4s', 'StarTAC 75', 'C200',
       'C139', 'Galaxy S7 & Galaxy S7 edge', '3100(3120)', '6230(6233)',
       'iPhone 4', '6300', 'Galaxy S6 & Galaxy S6 edge', 'N70(N72/N73)',
       'Galaxy S8 & Galaxy S8+', 'Galaxy S II', '2650', 'iPhone 3GS',
       'E250', 'KP100', 'Galaxy Note II N7100', '5200(5300)', 'Galaxy S',
       'Galaxy Grand Prime Plus', 'Chocolate VX8500', '2100',
       'Mate 10 & Mate 10 Pro', 'ThunderBolt 4G', 'Mate 9',
       '5610 XpressMusic', '5800 XpressMusic', 'E71', 'K300(K310)',
       'K750', 'W800(W810)', 'BlackBerry Pearl', 'DROID BION

### Exact matching by Model between the two datasets

In [40]:
common_values = pd.merge(df1[['model']], df2_before[['Model']], left_on='model', right_on='Model')['model'].unique()
print(len(common_values))

46


In [41]:
df2_filtered = df2_before[df2_before['Model'].isin(common_values)]
df1_filtered = df1[df1['model'].isin(common_values)]

df_before_merged = pd.merge(df2_filtered[['Model','Units Sold (million )','Manufacturer','Year']], df1_filtered, 
                       left_on=['Model', 'Manufacturer'], right_on=['model', 'brand'], how='inner')
df_before_merged

,Model,Units Sold (million ),Manufacturer,Year,approx_price_eur_scale,battery_mah_scale,ram_in_gb_scale,internal_memory_in_gb_scale,model,brand,id
0,1100,250.0,Nokia,2003,NaN,0.316401,NaN,NaN,1100,Nokia,Nokia 1100
1,1110,247.5,Nokia,2005,-0.322661,0.135943,NaN,-0.147863,1110,Nokia,Nokia 1110
2,iPhone 5s,164.5,Apple,2013,0.355742,-0.422488,0.059124,4.269950,iPhone 5s,Apple,Apple iPhone 5s
3,3210,161.0,Nokia,1999,NaN,NaN,NaN,NaN,3210,Nokia,Nokia 3210
4,6600,150.0,Nokia,2003,NaN,0.316401,NaN,-0.405369,6600,Nokia,Nokia 6600
5,1200,150.0,Nokia,2007,-0.910545,-0.894430,NaN,-0.209452,1200,Nokia,Nokia 1200
6,5230,150.0,Nokia,2009,-0.169966,1.079920,-0.772848,-0.260051,5230,Nokia,Nokia 5230
7,E1100,150.0,Samsung,2009,-0.279908,-1.124388,NaN,-0.275858,E1100,Samsung,Samsung E1100
8,iPhone 5,146.2,Apple,2012,1.853732,-0.210500,0.651283,4.664849,iPhone 5,Apple,Apple iPhone 5
9,1280,100.0,Nokia,2010,-0.317296,-0.818520,NaN,NaN,1280,Nokia,Nokia 1280


df_before_merged is the dataframe that contains the phones found in both data sets for phone before and in 2017. 

## Phone models after 2017

In [42]:
# Keep rows where 'Year' is greater than 2017
df2_after = df2[df2['Year'] > 2017].copy()

# reset index
df2_after.reset_index(drop=True, inplace=True)

# Display the first few rows to verify
(df2_after)

,Manufacturer,Model,Units Sold (million ),Rank,Year
0,Apple,"iPhone11, 11Pro & 11Pro Max",159.2,9,2019
1,Apple,"iPhone XR, iPhone XS & iPhone XS Max",151.1,10,2018
2,Apple,"iPhone 12, 12 mini ,12 Pro & 12 Pro Max",100.0,21,2020
3,Samsung,"Galaxy S10, Galaxy S10+ & Galaxy S10e",37.0,42,2019
4,Samsung,Galaxy S9 & Galaxy S9+,35.4,43,2018
5,Huawei,"P20, P20 Pro & P20 Lite",32.0,46,2018
6,Samsung,Galaxy A10,30.3,47,2019
7,Xiaomi,Redmi Note 8 & Redmi Note 8 Pro,30.0,52,2019
8,Apple,iPhone 12 Pro Max,29.0,53,2020
9,Samsung,"Galaxy S20, Galaxy S20+ & Galaxy S20 Ultra",28.0,54,2020


### Join Model and Manufacturer, and use RegEx to align with df1['id']

In [43]:
import re

# Apply additional regex replacements
df2_after['Model'] = df2_after['Model'].apply(lambda part: re.sub(r'(\d)(?=Pro)', r'\1 ', part))  # Add space before 'Pro'
df2_after['Model'] = df2_after['Model'].apply(lambda part: re.sub(r'(Note)(\d)', r'\1 \2', part))  # Add space between 'Note' and number
df2_after['Model'] = df2_after['Model'].apply(lambda part: re.sub(r'(iPhone)(\d)', r'\1 \2', part))  # Add space after 'iPhone'

In [44]:
import re

df2_after['id'] = df2_after.apply(
    lambda row: row['Manufacturer'] + ' ' + re.sub(rf'^{row["Manufacturer"]}\s*', '', row['Model'], flags=re.IGNORECASE), 
    axis=1
)

In [45]:
# Use regex to add '5G' only to the specified models
df2_after['id'] = df2_after['id'].str.replace(
    r'^(Samsung Galaxy S21(?:\+| Ultra)?|Samsung Galaxy S22 Ultra)$',
    r'\1 5G', regex=True)

### Exact matching

In [46]:
common_values = pd.merge(df1[['id']], df2_after[['id']], left_on='id', right_on='id')['id'].unique()
print(len(common_values))

25


In [47]:
df2_filtered = df2_after[df2_after['id'].isin(common_values)]
df1_filtered = df1[df1['id'].isin(common_values)]

df_after_merged = pd.merge(df2_filtered[['Units Sold (million )','id','Year']], df1_filtered.drop(['model', 'brand'], axis=1), 
                       left_on=['id'], right_on=['id'], how='inner')
df_after_merged

,Units Sold (million ),id,Year,approx_price_eur_scale,battery_mah_scale,ram_in_gb_scale,internal_memory_in_gb_scale
0,30.3,Samsung Galaxy A10,2019,-0.736397,-0.295214,-0.948828,-0.667524
1,29.0,Apple iPhone 12 Pro Max,2020,0.475788,-0.661448,0.264498,0.397905
2,24.2,Samsung Galaxy A50,2019,0.083795,0.221625,-0.136165,-0.180109
3,23.2,Samsung Galaxy A51,2019,-0.099001,0.221625,-0.136165,-0.180109
4,23.1,Samsung Galaxy A20,2019,-0.451113,0.221625,-0.542497,-0.667524
5,19.4,Samsung Galaxy A21s,2020,-0.431267,0.768138,-0.978546,-1.003751
6,16.9,Samsung Galaxy A01,2019,-0.655452,-1.409450,-1.392895,-1.237360
7,15.3,Samsung Galaxy A11,2020,-0.514019,-0.320656,-1.392895,-1.003751
8,15.2,Samsung Galaxy J2 Core,2018,-0.927901,-0.817617,-1.385318,-1.060132
9,15.0,Xiaomi Redmi Note 9 Pro,2020,-0.289513,0.789914,0.264498,-0.536532


# Merged dataframes from before and after 2017 into one dataframe

Include only relevant columns

In [48]:
df_before_merged = df_before_merged.drop(['Model','Manufacturer','model', 'brand'], axis=1)
df_before_merged.shape

(44, 7)

In [49]:
df_combined = pd.concat([df_before_merged, df_after_merged], ignore_index=True)

In [50]:
df_combined = df_combined.rename(columns={'Units Sold (million )': 'sales'})

In [51]:
df_combined

,sales,Year,approx_price_eur_scale,battery_mah_scale,ram_in_gb_scale,internal_memory_in_gb_scale,id
0,250.0,2003,NaN,0.316401,NaN,NaN,Nokia 1100
1,247.5,2005,-0.322661,0.135943,NaN,-0.147863,Nokia 1110
2,164.5,2013,0.355742,-0.422488,0.059124,4.269950,Apple iPhone 5s
3,161.0,1999,NaN,NaN,NaN,NaN,Nokia 3210
4,150.0,2003,NaN,0.316401,NaN,-0.405369,Nokia 6600
...,...,...,...,...,...,...,...
64,3.9,2019,-0.561660,0.221625,-0.948828,-0.667524,Samsung Galaxy A10s
65,3.8,2023,-0.132891,-0.559350,-0.078946,-0.485727,Samsung Galaxy S23 FE
66,3.4,2019,-0.395482,0.221625,-0.542497,-0.667524,Samsung Galaxy A30s
67,2.7,2024,-0.813792,-0.202063,-1.255736,-0.925027,Samsung Galaxy A55


# Handle missing values

In [52]:
df_combined.isna().sum()

sales                           0
Year                            0
approx_price_eur_scale          8
battery_mah_scale               1
ram_in_gb_scale                24
internal_memory_in_gb_scale     9
id                              0
dtype: int64

In [53]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

def preprocess_data(df, numerical_columns, categorical_columns, n_neighbors=3):
    """
    Preprocesses a DataFrame by imputing missing numerical values and keeping categorical columns unchanged.
    
    Parameters:
        df (pd.DataFrame): The input DataFrame.
        numerical_columns (list): List of numerical columns to be imputed.
        categorical_columns (list): List of categorical columns to be passed through.
        n_neighbors (int): Number of neighbors for KNN imputation (default=3).
    
    Returns:
        pd.DataFrame: Transformed DataFrame with imputed numerical columns and unchanged categorical columns.
    """
    # Define the KNN imputer for numerical data
    numerical_imputer = KNNImputer(n_neighbors=n_neighbors)

    # Define the column transformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('numerical', numerical_imputer, numerical_columns),
            ('categorical', 'passthrough', categorical_columns)
        ]
    )

    # Apply transformations
    transformed_data = preprocessor.fit_transform(df)

    # Convert back to a DataFrame
    transformed_df = pd.DataFrame(transformed_data, columns=numerical_columns + categorical_columns)

    return transformed_df


In [54]:
# Define column lists
numerical_columns = ['approx_price_eur_scale', 'battery_mah_scale', 'ram_in_gb_scale',
                     'internal_memory_in_gb_scale', 'sales']
categorical_columns = ['Year', 'id']

# Call the function
df_imputed = preprocess_data(df_combined, numerical_columns, categorical_columns)

df_imputed

,approx_price_eur_scale,battery_mah_scale,ram_in_gb_scale,internal_memory_in_gb_scale,sales,Year,id
0,-0.045628,0.316401,-0.020814,1.238906,250.0,2003,Nokia 1100
1,-0.322661,0.135943,-0.020814,-0.147863,247.5,2005,Nokia 1110
2,0.355742,-0.422488,0.059124,4.26995,164.5,2013,Apple iPhone 5s
3,-0.031377,-0.155652,-0.020814,1.24468,161.0,1999,Nokia 3210
4,-0.453473,0.316401,-0.020814,-0.405369,150.0,2003,Nokia 6600
...,...,...,...,...,...,...,...
64,-0.56166,0.221625,-0.948828,-0.667524,3.9,2019,Samsung Galaxy A10s
65,-0.132891,-0.55935,-0.078946,-0.485727,3.8,2023,Samsung Galaxy S23 FE
66,-0.395482,0.221625,-0.542497,-0.667524,3.4,2019,Samsung Galaxy A30s
67,-0.813792,-0.202063,-1.255736,-0.925027,2.7,2024,Samsung Galaxy A55


In [55]:
df_imputed.isna().sum()

approx_price_eur_scale         0
battery_mah_scale              0
ram_in_gb_scale                0
internal_memory_in_gb_scale    0
sales                          0
Year                           0
id                             0
dtype: int64

# Scale sales column

In [56]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

def scale_group(group):
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(group[numerical_cols])
    scaled_cols = [col + '_scale' for col in numerical_cols]
    scaled_df = pd.DataFrame(scaled_data, columns=scaled_cols, index=group.index)
    for orig_col, scaled_col in zip(numerical_cols, scaled_cols):
        
        # Insert scaled columns right after the original columns
        col_idx = group.columns.get_loc(orig_col)
        group.insert(col_idx + 1, scaled_col, scaled_df[scaled_col])
    return group

In [57]:
# Copy the dataframe for safety
df = df_imputed.copy(deep=True)

# Define the column to scale
numerical_cols = ['sales']
group_col = 'Year'

# Apply scaling to each group by the specified column
df_transformed = df.groupby(group_col, group_keys=False).apply(scale_group)

# Sort by 'year' and other identifiers if needed
df_transformed = df_transformed.sort_values(['Year', 'id'])

# remove sales
df_transformed = df_transformed.drop('sales', axis=1)

(df_transformed)

,approx_price_eur_scale,battery_mah_scale,ram_in_gb_scale,internal_memory_in_gb_scale,sales_scale,Year,id
13,0.133253,-0.437476,0.377943,3.479012,0.000000,1996,Motorola StarTAC 75
3,-0.031377,-0.155652,-0.020814,1.24468,0.000000,1999,Nokia 3210
14,0.133253,-0.67555,0.377943,3.479012,-0.397680,2003,Motorola C200
0,-0.045628,0.316401,-0.020814,1.238906,1.666777,2003,Nokia 1100
24,-0.512611,-0.54329,-0.971313,-0.969545,-0.832302,2003,Nokia 2100
...,...,...,...,...,...,...,...
54,-0.010106,0.180955,0.36645,-0.157614,0.000000,2022,Samsung Galaxy S22 Ultra 5G
65,-0.132891,-0.55935,-0.078946,-0.485727,0.000000,2023,Samsung Galaxy S23 FE
67,-0.813792,-0.202063,-1.255736,-0.925027,-0.568241,2024,Samsung Galaxy A55
68,-0.298376,-1.664553,-0.701119,-0.925027,-0.837408,2024,Samsung Galaxy S24


# Save the final DataFrame to a CSV file

In [58]:
# Save the final DataFrame to a CSV file
df_transformed.to_csv('../2.Hierarchical_clustering/df_transformed.csv', index=False)